In [1]:
from evaluation.generated_dataset import load_WyCryst_csv, GeneratedDataset

In [2]:
wtt=GeneratedDataset.from_transformations(("WyCryst",), dataset="mp_20_biternary")

In [3]:
wtt.data.head()

,species,sites,numIons,group,site_symmetries,elements,multiplicity,wyckoff_letters,sites_enumeration,spacegroup_number,sites_enumeration_augmented,corrected_chgnet_ehull
0,"[Nd, Al, Cu]","[[2a], [8i, 8j], [8f]]","[2, 16, 8]",139,"[4/mmm, m2m., m2m., ..2/m]","[Nd, Al, Al, Cu]","[2, 8, 8, 8]","[a, i, j, f]","[0, 0, 1, 0]",139,"((1, 1, 0, 0), (0, 0, 1, 0))",-0.003829
1,"[Li, Mn, Ir]","[[4b], [4a], [8c]]","[4, 4, 8]",225,"[m-3m, m-3m, -43m]","[Li, Mn, Ir]","[4, 4, 8]","[b, a, c]","[1, 0, 0]",225,"((1, 0, 0), (0, 1, 0))",-0.000484
4,"[Na, Bi, O]","[[2a, 2d, 4h], [4g, 4i], [4i, 8j]]","[8, 8, 12]",12,"[2/m, 2/m, 2, 2, m, m, 1]","[Na, Na, Na, Bi, Bi, O, O]","[2, 2, 4, 4, 4, 4, 8]","[a, d, h, g, i, i, j]","[0, 3, 1, 0, 0, 0, 0]",12,"((0, 3, 1, 0, 0, 0, 0), (3, 0, 0, 1, 0, 0, 0),...",0.032536
5,"[In, Bi, S]","[[2a, 2e], [2e, 2e], [2e, 2e, 2e]]","[4, 4, 6]",11,"[-1, m, m, m, m, m, m]","[In, In, Bi, Bi, S, S, S]","[2, 2, 2, 2, 2, 2, 2]","[a, e, e, e, e, e, e]","[0, 0, 0, 0, 0, 0, 0]",11,"((3, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0),...",-0.003587
7,"[Ti, Cu]","[[4c], [4c, 4c, 4c, 4c]]","[4, 16]",62,"[.m., .m., .m., .m., .m.]","[Ti, Cu, Cu, Cu, Cu]","[4, 4, 4, 4, 4]","[c, c, c, c, c]","[0, 0, 0, 0, 0]",62,"((0, 0, 0, 0, 0))",-0.011174


In [2]:
crystalformer_raw = GeneratedDataset()
crystalformer_raw.load_structures("generated/CrystalFormer_mp_20.csv.gz", StructureStorage.CrystalFormer)

In [3]:
crystalformer_raw.compute_wyckoffs(n_jobs=20)

In [4]:
crystalformer_raw.convert_wyckoffs_to_pyxtal()


In [5]:
crystalformer_raw.compute_cdvae_crystals(n_jobs=20)

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.10/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:4158: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.10/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:4158: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.10/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:4158: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.10/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:3951: UserWarning: CrystalNN: cannot

In [6]:
crystalformer_raw.data.iloc[0]

structure                      [[5.03535423 5.03535423 5.03535423] Cu, [5.035...
site_symmetries                                      [-43m, -43m, .3m, .3m, .3m]
elements                                                     [Cu, Cu, S, Se, Cr]
multiplicity                                                  [4, 4, 16, 16, 16]
wyckoff_letters                                                  [b, c, e, e, e]
sites_enumeration                                                [1, 2, 0, 0, 0]
dof                                                              [0, 0, 1, 1, 1]
spacegroup_number                                                            216
sites_enumeration_augmented    ((2, 0, 0, 0, 0), (1, 3, 0, 0, 0), (0, 3, 0, 0...
composition                                       {Cu: 8, S: 16, Se: 16, Cr: 16}
group                                                                        216
sites                                            [[4b, 4c], [16e], [16e], [16e]]
species                     

In [7]:
from typing import Dict
from pandas import Series
def record_to_augmented_fingerprint(row: Dict|Series) -> tuple:
    """
    Computes a fingerprint includeing possible Wyckoff position enumeration.
    Args:
        row contains the Wyckoff information:
        - spacegroup_number
        - elements
        - site_symmetries
        - sites_enumeration_augmented
    Returns:
        A tuple of the spacegroup number and a frozenset of tuples of the elements,
        site symmetries and the Wyckoff position enumeration.
    """
    spacegroup_number = row["spacegroup_number"]
    transposed_augmentations = zip(*row["sites_enumeration_augmented"])
    return (
        row["spacegroup_number"],
        frozenset(
            map(
                tuple,
                zip(row["elements"], row["site_symmetries"], *transposed_augmentations)
            )
        )
    )

In [18]:
mp_20_test_fingerprints = mp_20_test.data.apply(record_to_augmented_fingerprint, axis=1)
crystalformer_raw_fingerprints = crystalformer_raw.data.apply(record_to_augmented_fingerprint, axis=1)

In [20]:
len(frozenset(mp_20_test_fingerprints))/len(mp_20_test_fingerprints)

0.8841476895865575

In [13]:
len(crystalformer_raw_fingerprints)

999

In [24]:
len(frozenset(crystalformer_raw_fingerprints) - frozenset(mp_20_test_fingerprints))/len(frozenset(crystalformer_raw_fingerprints))

0.9579055441478439